In [2]:
import urllib.request

# 下載shape_predictor_68_face_landmarks.dat模型文件
url = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"
save_path = "shape_predictor_68_face_landmarks.dat.bz2"

urllib.request.urlretrieve(url, save_path)

# 解壓縮模型文件
import bz2
import shutil

uncompressed_path = "shape_predictor_68_face_landmarks.dat"

with open(uncompressed_path, 'wb') as output, bz2.BZ2File(save_path, 'rb') as input:
    shutil.copyfileobj(input, output)

# 刪除下載的壓縮文件
import os

os.remove(save_path)

print("模型文件下載完成並解壓縮。")


模型文件下載完成並解壓縮。


In [3]:
import cv2
import dlib
import math
import os

# 初始化dlib的人臉偵測器
detector = dlib.get_frontal_face_detector()

# 初始化dlib的人臉關鍵點偵測器
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# 設定眼睛開合程度的閾值
THRESHOLD = 0.3

# 計算眼睛的開合程度
def eye_aspect_ratio(eye_landmarks):
    # 計算垂直眼睛關鍵點之間的歐氏距離
    vertical_dist1 = math.sqrt((eye_landmarks[1].x - eye_landmarks[5].x) ** 2 + (eye_landmarks[1].y - eye_landmarks[5].y) ** 2)
    vertical_dist2 = math.sqrt((eye_landmarks[2].x - eye_landmarks[4].x) ** 2 + (eye_landmarks[2].y - eye_landmarks[4].y) ** 2)
    
    # 計算水平眼睛關鍵點之間的歐氏距離
    horizontal_dist = math.sqrt((eye_landmarks[0].x - eye_landmarks[3].x) ** 2 + (eye_landmarks[0].y - eye_landmarks[3].y) ** 2)
    
    # 計算眼睛的開合程度
    ear = (vertical_dist1 + vertical_dist2) / (2.0 * horizontal_dist)
    
    return ear

# 設定圖片目錄的路徑
image_dir = "/content/drive/MyDrive/Image recognition"

# 設定結果保存目錄的路徑
output_dir = "/content/drive/MyDrive/Image recognition2"

# 建立結果保存目錄
os.makedirs(output_dir, exist_ok=True)

# 迴圈遍歷圖片目錄中的所有圖片
for filename in os.listdir(image_dir):
    # 檢查檔案是否為圖片格式
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # 讀取影像
        image_path = os.path.join(image_dir, filename)
        image = cv2.imread(image_path)

        # 將影像轉換為灰階
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # 人臉偵測
        faces = detector(gray)

        # 迭代偵測到的人臉
        for face in faces:
            # 人臉關鍵點偵測
            landmarks = predictor(gray, face)

            # 提取左眼和右眼的關鍵點座標
            left_eye_landmarks = landmarks.parts()[36:42]  # 左眼關鍵點索引為36-41
            right_eye_landmarks = landmarks.parts()[42:48]  # 右眼關鍵點索引為42-47

            # 計算眼睛的開合程度
            left_eye_aspect_ratio = eye_aspect_ratio(left_eye_landmarks)
            right_eye_aspect_ratio = eye_aspect_ratio(right_eye_landmarks)

            # 判斷是否為睡覺狀態
            if left_eye_aspect_ratio < THRESHOLD and right_eye_aspect_ratio < THRESHOLD:
                cv2.putText(image, "Sleeping", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                cv2.putText(image, "Awake", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # 儲存結果影像
        output_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_path, image)
